In [1]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning

warnings.filterwarnings('ignore', category=UndefinedMetricWarning)
warnings.filterwarnings('ignore', category=UserWarning)

In [ ]:
import numpy as np
import pandas as pd
from rdkit import Chem
from skfp.fingerprints import (
    PubChemFingerprint, LaggnerFingerprint, AvalonFingerprint,
    AtomPairFingerprint, ECFPFingerprint, MACCSFingerprint
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from skmultilearn.adapt import MLkNN
#from sklearn.model_selection import KFold
from sklearn.metrics import (
    roc_auc_score, average_precision_score,
    precision_score, recall_score
)
from tqdm import tqdm

import deepchem as dc
import umap

In [3]:
smiles_df = pd.read_csv('smiles.csv', header=None)
smiles_list = smiles_df.iloc[:, 0].tolist()
Y_lltpt = pd.read_csv('/home/maciej/studia/praktyki_ibb/lltpt_binary.csv').to_numpy()
Y_hlt = pd.read_csv('/home/maciej/studia/praktyki_ibb/hlt_binary.csv').to_numpy()
Y_hlgt = pd.read_csv('/home/maciej/studia/praktyki_ibb/hlgt_binary.csv').to_numpy()


In [4]:
smiles_list_clean = []
idx_clean = []
mols_clean = []
for i, smi in enumerate(smiles_list):
    mol = Chem.MolFromSmiles(smi)
    if mol is not None:
        smiles_list_clean.append(smi)
        mols_clean.append(mol)
        idx_clean.append(i)
Y_lltpt_clean = Y_lltpt[idx_clean, :]
Y_hlt_clean = Y_hlt[idx_clean, :]
Y_hlgt_clean = Y_hlgt[idx_clean, :]

In [5]:
countEnabled = True
njobs_=5

In [6]:
fingerprints = {
    'PubChem': PubChemFingerprint(n_jobs=njobs_, sparse=False, count=countEnabled),
    'Laggner': LaggnerFingerprint(n_jobs=njobs_, sparse=False, count=countEnabled),
    'Avalon': AvalonFingerprint(n_jobs=njobs_, sparse=False, count=countEnabled),
    'AtomPair': AtomPairFingerprint(n_jobs=njobs_, sparse=False, count=countEnabled),
    'ECFP': ECFPFingerprint(n_jobs=njobs_, sparse=False, count=countEnabled),
    'MACCS': MACCSFingerprint(n_jobs=njobs_, sparse=False, count=countEnabled),
}
fp_dims = {
    'PubChem': 881,
    'Laggner': 307,
    'Avalon': 512,
    'AtomPair': 2048,
    'ECFP': 2048,
    'MACCS': 166,
}

In [15]:
param_grids = {
    'RandomForest': {
        'n_estimators': [100],
        'max_depth': [None],
        'min_samples_split': [2, 4,8, 10],
        'min_samples_leaf': [2, 4, 6]
    },
    'KNN': {
        'n_neighbors': [3, 5, 7, 11, 17],
        'weights': ['uniform', 'distance'],
        'p': [1, 2],  
    },
    'MLkNN': {
        'k': [3, 5, 7, 10],
        's': [0.1, 0.5, 1.0, 1.5]
    }
}

param_grid_mlknn = {
        'k': [3, 5, 7, 10],
        's': [0.1, 0.5, 1.0, 1.5]
}



In [16]:
from sklearn.model_selection import GridSearchCV

def run_grid_search(model, param_grid, X_train, Y_train):
    """
    perform grid search cross-validation

    uses GridSearchCV from sklearn

    return
        best params and best score
    """
    grid_search = GridSearchCV(model, param_grid, scoring='roc_auc_ovr', cv=3, n_jobs=njobs_, verbose=1)
    grid_search.fit(X_train, Y_train)
    return grid_search.best_params_, grid_search.best_score_


In [17]:
models = {
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=njobs_),
    'KNN': KNeighborsClassifier(n_neighbors=5, n_jobs=njobs_),
    'MLkNN': MLkNN(k=10, s=1.0)
}
levels = [
    ('LLT/PT', Y_lltpt_clean),
    ('HLT', Y_hlt_clean),
    ('HLGT', Y_hlgt_clean)
]

In [18]:
def get_proba_multilabel(model, X, Y):
    if hasattr(model, "predict_proba"):
        Y_pred = model.predict_proba(X)
        if isinstance(Y_pred, list) or isinstance(Y_pred, tuple):
            proba = np.vstack([p[:, 1] if p.shape[1] == 2 else np.zeros(p.shape[0]) for p in Y_pred]).T
        else:
            proba = Y_pred.toarray() if hasattr(Y_pred, "toarray") else Y_pred
        return proba
    else:
        return model.predict_proba(X)


In [19]:
def run_w_scaffold(model, X_train, Y_train, X_test, Y_test, multilabel=False):
    """
    train model on train set, evaluate on test set, compute metrics

    supports multiclass models

    return
        dictionary of metrics
    """
    model.fit(X_train, Y_train)
    if multilabel:
        Y_pred_proba = get_proba_multilabel(model, X_test, Y_test)
    else:
        Y_pred = model.predict_proba(X_test)
        proba_list = []
        for i, p in enumerate(Y_pred):
            if p.shape[1] == 2:
                proba_list.append(p[:,1])
            else:
                present_class = model.classes_[i][0]
                if present_class == 1:
                    proba_list.append(p[:, 0])
                else:
                    proba_list.append(np.zeros_like(p[:, 0]))
        Y_pred_proba = np.vstack(proba_list).T

    
    auc_micro = roc_auc_score(Y_test, Y_pred_proba, average="micro") * 100
    aupr_micro = average_precision_score(Y_test, Y_pred_proba) * 100
    try:
        auc_macro = roc_auc_score(Y_test, Y_pred_proba, average='macro') * 100
    except Exception:
        auc_macro = float('nan')
    try:
        aupr_macro = average_precision_score(Y_test, Y_pred_proba, average='macro') * 100
    except Exception:
        aupr_macro = float('nan')
    Y_pred_bin = (Y_pred_proba >= 0.5).astype(int)
    precision_micro = precision_score(Y_test, Y_pred_bin, average='micro', zero_division=0) * 100
    recall_micro = recall_score(Y_test, Y_pred_bin, average='micro', zero_division=0) * 100

    return {
        'auc_micro': auc_micro,
        'aupr_micro': aupr_micro,
        'auc_macro': auc_macro,
        'aupr_macro': aupr_macro,
        'precision_micro': precision_micro,
        'recall_micro': recall_micro
    }

In [20]:
import numpy as np
from sklearn.metrics import roc_auc_score


def safe_roc_auc(estimator, X, y):
    """
    compute multilabel roc-auc for models with diverse output formats

    return 
        macro-avg roc-auc (or nan for failure)
    """
    if hasattr(estimator, "predict_proba"):
        y_pred = estimator.predict_proba(X)
        if isinstance(y_pred, list):
            result = []
            n_samples = X.shape[0]
            for p in y_pred:
                
                if p.ndim == 2 and p.shape[1] == 2:
                    result.append(p[:, 1])
                
                elif p.ndim == 2 and p.shape[1] == 1:
                    result.append(np.zeros(n_samples))
                
                elif p.ndim == 1 and p.shape[0] == n_samples:
                    result.append(p)
                else:
                    result.append(np.zeros(n_samples))
            y_pred = np.vstack(result).T  # shape: (n_samples, n_labels)
        elif hasattr(y_pred, "toarray"):
            y_pred = y_pred.toarray()
    else:
        y_pred = estimator.predict(X)
        if hasattr(y_pred, "toarray"):
            y_pred = y_pred.toarray()
        if isinstance(y_pred, list):
            y_pred = np.array(y_pred)

    y_pred = np.asarray(y_pred)
    y = np.asarray(y)

    try:
        
        return roc_auc_score(y, y_pred, average="macro")
    except Exception:
        return np.nan        

In [ ]:



results = []
X_cache = {}
param_file = "best_params_all_biggergrid.txt"
open(param_file, "w").close()  

for fp_name, fp in tqdm(fingerprints.items(), desc="Fingerprints"):
    print(f"=== Fingerprint: {fp_name} ===")
    if fp_name not in X_cache:
        X = fp.fit_transform(smiles_list_clean)
        if hasattr(X, "toarray"):
            X = X.toarray()
        X_cache[fp_name] = X
    else:
        X = X_cache[fp_name]

    dataset = dc.data.NumpyDataset(X, y=None, ids=smiles_list_clean)
    splitter = dc.splits.ScaffoldSplitter()
    train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(
        dataset, frac_train=0.8, frac_valid=0.0, frac_test=0.2
    )
    train_idx = [np.where(dataset.ids == id)[0][0] for id in train_dataset.ids]
    test_idx = [np.where(dataset.ids == id)[0][0] for id in test_dataset.ids]

    X_train = X[train_idx]
    X_test = X[test_idx]

    umap_model = umap.UMAP(
        n_neighbors=15,
        min_dist=0.1,
        n_components=2,
        metric='euclidean',
        random_state=42,
        n_epochs=500,
        n_jobs=-1
    )
    X_train_umap = umap_model.fit_transform(X_train)
    X_test_umap = umap_model.transform(X_test)

    for level_name, Y in levels:
        Y_train = np.asarray(Y[train_idx])
        Y_test = np.asarray(Y[test_idx])
        for model_name, model in models.items():
            print(f" - {model_name} on {level_name}")

            if model_name in param_grids:
                estimator = model if model_name != 'MLkNN' else MLkNN()
                grid_search = GridSearchCV(
                    estimator,
                    param_grids[model_name],
                    scoring=safe_roc_auc,
                    cv=3,
                    n_jobs=1,
                    verbose=1,
                    pre_dispatch='2*n_jobs',
                    error_score='raise'
                )
                grid_search.fit(X_train_umap, Y_train)
                best_model = grid_search.best_estimator_
                best_params = grid_search.best_params_
                best_score = grid_search.best_score_
                print(f"   Best params: {best_params}, best score: {best_score:.4f}")

                with open(param_file, "a") as f:
                    f.write(f"Fingerprint: {fp_name}, Model: {model_name}, Level: {level_name}\n")
                    f.write(f"Best params: {best_params}\n")
                    f.write(f"Best score: {best_score:.4f}\n\n")

                multilabel = (model_name == 'MLkNN')
                res = run_w_scaffold(best_model, X_train_umap, Y_train, X_test_umap, Y_test, multilabel=multilabel)
            else:
                multilabel = (model_name == 'MLkNN')
                res = run_w_scaffold(model, X_train_umap, Y_train, X_test_umap, Y_test, multilabel=multilabel)

            results.append({
                'fingerprint': fp_name,
                'model': model_name,
                'level': level_name,
                'auc_micro': res['auc_micro'],
                'aupr_micro': res['aupr_micro'],
                'auc_macro': res['auc_macro'],
                'aupr_macro': res['aupr_macro'],
                'precision_micro': res['precision_micro'],
                'recall_micro': res['recall_micro'],
                'dim': fp_dims[fp_name]
            })


In [ ]:
filename = "results_all_umap.txt"
filepath = f"results/{filename}"



In [ ]:
with open(filepath, "w") as f:
    for r in results:
        f.write(
            f"{r['fingerprint']}/{r['model']}/{r['level']}/"
            f"AUC_micro:{r['auc_micro']:.2f} "
            f"AUPR_micro:{r['aupr_micro']:.2f} "
            f"AUC_macro:{r['auc_macro']:.2f} "
            f"AUPR_macro:{r['aupr_macro']:.2f} "
            f"Precision_micro:{r['precision_micro']:.2f} "
            f"Recall_micro:{r['recall_micro']:.2f} "
            f"Dim:{r['dim']}\n"
        )
print(f"saved as {filename}")


In [ ]:
from make_table import parse_metrics, generate_text_table, generate_latex_table


with open(filepath, 'r') as f:
    file_content = f.read()






metrics_df = parse_metrics(file_content)


metrics_df


,dataset,model,level,AUC_micro,AUPR_micro,AUC_macro,AUPR_macro,Precision_micro,Recall_micro,Dim
0,PubChem,RandomForest,LLT,76.19,28.81,None,12.24,42.45,21.16,881
1,PubChem,KNN,LLT,69.00,21.06,None,10.97,38.81,23.13,881
2,PubChem,MLkNN,LLT,79.76,33.53,None,9.71,88.34,0.59,881
3,PubChem,RandomForest,HLT,79.75,41.97,None,18.19,52.87,26.08,881
4,PubChem,KNN,HLT,76.10,37.55,None,18.19,47.69,29.04,881
5,PubChem,MLkNN,HLT,80.20,41.58,None,14.46,89.41,0.78,881
6,PubChem,RandomForest,HLGT,79.64,50.91,None,23.51,54.48,41.66,881
7,PubChem,KNN,HLGT,74.75,41.39,None,22.70,52.30,42.12,881
8,PubChem,MLkNN,HLGT,80.82,53.79,None,20.95,90.86,2.51,881
9,Laggner,RandomForest,LLT,76.92,30.89,None,13.06,45.08,21.35,307


In [ ]:

print(generate_latex_table(metrics_df))


\begin{tabular}{lllrrlrrrr}
\toprule
dataset & model & level & AUC_micro & AUPR_micro & AUC_macro & AUPR_macro & Precision_micro & Recall_micro & Dim \\
\midrule
PubChem & RandomForest & LLT & 76.19 & 28.81 & NaN & 12.24 & 42.45 & 21.16 & 881 \\
PubChem & KNN & LLT & 69.00 & 21.06 & NaN & 10.97 & 38.81 & 23.13 & 881 \\
PubChem & MLkNN & LLT & 79.76 & 33.53 & NaN & 9.71 & 88.34 & 0.59 & 881 \\
PubChem & RandomForest & HLT & 79.75 & 41.97 & NaN & 18.19 & 52.87 & 26.08 & 881 \\
PubChem & KNN & HLT & 76.10 & 37.55 & NaN & 18.19 & 47.69 & 29.04 & 881 \\
PubChem & MLkNN & HLT & 80.20 & 41.58 & NaN & 14.46 & 89.41 & 0.78 & 881 \\
PubChem & RandomForest & HLGT & 79.64 & 50.91 & NaN & 23.51 & 54.48 & 41.66 & 881 \\
PubChem & KNN & HLGT & 74.75 & 41.39 & NaN & 22.70 & 52.30 & 42.12 & 881 \\
PubChem & MLkNN & HLGT & 80.82 & 53.79 & NaN & 20.95 & 90.86 & 2.51 & 881 \\
Laggner & RandomForest & LLT & 76.92 & 30.89 & NaN & 13.06 & 45.08 & 21.35 & 307 \\
Laggner & KNN & LLT & 68.84 & 21.51 & NaN & 11.